In [13]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
  #raise SystemError('GPU device not found')    
else:
  print('I Found a GPU! at: {}'.format(device_name))


import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
#import librosa
import sklearn
import pandas as pd
from tensorflow import keras
from keras.models import load_model
from keras import layers
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Reshape , Dropout
from keras.regularizers import l1
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
plt.style.use('seaborn')

GPU device not found


C:\Users\david\AppData\Local\Temp\ipykernel_5908\193200553.py:24: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


### Defining the Training and Target set
 
To define the target set, it is necessary to ensure that its first dimension matches that of the training set (the number of rows). We have a training set of shape(num_samples,1, 8), it means with 8 features each wiht dimension 1. We want to have a 5 floats outputs so we need to define a target set of num_sample rows and 5 targets ( 5 output features). 

In [2]:
#completare con valori sensati

#Training set 
pd_dataframe= {
    'arpeggio': pd.read_csv('csv_pd_dataframe\df_arpeggio_rms'),
    'strumming': pd.read_csv('csv_pd_dataframe/df_strumming_rms'),
    'bending': pd.read_csv('csv_pd_dataframe/df_bending_rms'),
    'double pick':pd.read_csv('csv_pd_dataframe/df_doublePick_rms'),
    'strong pick':pd.read_csv('csv_pd_dataframe/df_strongPick_rms'),
    'tapping':pd.read_csv('csv_pd_dataframe/df_tapping_rms'),
    'pullOff Hammer on': pd.read_csv('csv_pd_dataframe/df_pullOffHammerOn_rms')
}
#shaping all the pd dataset to a (30000, 8)
for i in pd_dataframe:
  pd_dataframe[i]= pd_dataframe[i].iloc[5000:35000].values # excluding the first and the last 5 second of samples
  pd_dataframe[i]= np.reshape(pd_dataframe[i],(pd_dataframe[i].shape[0], 1, pd_dataframe[i].shape[1]))
print('The lenght and shapes of my training datasets = {}, {}'.format(len(pd_dataframe), [pd_dataframe[key].shape for key in pd_dataframe]))

#Printing some statistics 
for i in pd_dataframe.keys():
  print('mean and variance of', i ,' = {:.3f}, {:.3f}'.format(np.mean(pd_dataframe[i]), np.var(pd_dataframe[i])))

#Target Set
target_dict = {
    'arpeggio': np.array([0.5,0.3, 0.2, 0.3, 0.5]),
    'strumming': np.array([0.6,0.2, 0.8, 0.7, 0.3]),
    'bending': np.array([0.2,0.1, 0.1, 0.5, 0.2]),
    'strongPick': np.array([0.4,0.5, 0.3, 0.4, 0.1]),
    'tapping': np.array([0.3,0.2, 0.6, 0.6, 0.7]),
    'pullOffHammerOn': np.array([0.7,0.7, 0.8, 0.2, 0.4]),
    'doublePick': np.array([0.8,0.4, 0.9, 0.1, 0.8]),
}
#populate the dictionaty with the right shapes before using them as target set
# i have to create a target set with 8 rows and 5 columns 
for i in target_dict:  
  target_dict[i]= np.tile(target_dict[i], (30000,1))

print("The Shape of the Target set: ",[i.shape for i in target_dict.values()])

The Shape of the Target set:  [(30000, 5), (30000, 5), (30000, 5), (30000, 5), (30000, 5), (30000, 5), (30000, 5)]
The lenght and shapes of my training datasets = 7, [(30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8)]
mean and variance of arpeggio  = 2236.176, 1305701.184
mean and variance of strumming  = 4943.854, 26904060.131
mean and variance of bending  = 3656.116, 151639304.146
mean and variance of double pick  = 2294.655, 2121090.473
mean and variance of strong pick  = 2088.049, 2635636.172
mean and variance of tapping  = 2379.778, 2249018.599
mean and variance of pullOff Hammer on  = 2526.514, 6551681.842


## Training dataset packing
Here I pack my input data to feed them to the Model during the training process. Furthemore we split the Training dataset in two part, one for the training process and one for the evalution, called Test set.

In [12]:

concatenated_training_datasets= np.concatenate(([pd_dataframe[key] for key in pd_dataframe.keys()]),axis=0)
#print(x_train.shape)
concatenated_target_datasets= np.concatenate(([target_dict[key] for key in target_dict.keys()]), axis= 0)
#print(y_train.shape)
#splitting the dataset into a training and testing dataset



X_train, X_test, Y_train, Y_test = train_test_split(concatenated_training_datasets, concatenated_target_datasets, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(168000, 1, 8) (42000, 1, 8) (168000, 5) (42000, 5)


## LSTM Sequential Network


In [8]:
model = Sequential()
#By setting the input shape to (8, 1), you are specifying that each sample consists of 8 time steps, each time step consisting of a single feature
# Add an input dense layer with 8 neurons, one for each column in the input dataset

# Add an LSTM layer with 64 units
model.add(LSTM(64, input_shape= (1,8),return_sequences=True, kernel_regularizer=l1(0.1))) # when you want to stack multiple LSTM 
#you have to set the return_sequences=True to not have shape problems. 

model.add(Dropout(0.3))  # add a dropout layer with dropout rate of 0.2
model.add(LSTM(64))
# Add a dense output layer with 5 units and linear activation
model.add(Dense(5, activation='linear')) #return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence. Default: False. IT should be set true only in the output layer 

print('The output shape: ',model.output_shape)
model.summary()

The output shape:  (None, 5)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 1, 64)             18688     
                                                                 
 dropout_3 (Dropout)         (None, 1, 64)             0         
                                                                 
 lstm_7 (LSTM)               (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
Total params: 52,037
Trainable params: 52,037
Non-trainable params: 0
_________________________________________________________________


## Compiling the Model
To compile the network (before training it) we have to choose the Optimizer and the loss function: 

The optimizer is the algorithm used to update the weights of the model during training, and 'adam' is a popular optimizer that adapts the learning rate of each weight based on the first and second moments of the gradients.

The loss function is used to measure how well the model is performing on the training data and guides the optimization process. 'mse' stands for mean squared error, which is a common loss function used for regression problems where the goal is to minimize the difference between the predicted and actual values.

In [ ]:
#Compiling the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])



## Training the Model

In [ ]:
history = model.fit(X_train,Y_train, validation_split=0.25, epochs=75)


The validation_split argument in Keras fit method specifies the fraction of the training data to be used as validation data. The model will not be trained on the validation data, but rather the validation data will be used to evaluate the model's training performance over the epochs on data it has not seen before.

For example, if validation_split=0.2, then 20% of the training data will be used as validation data and the remaining 80% will be used as the training data. The validation data will be used to evaluate the model's performance after each epoch during training. The validation set is a portion of the training set to evaluate the training process, instead to evaluate the network we have to use a different dataset called testing dataset.


### Plotting the perormance

We can track the training accurancy to avoid overfitting by plotting the loss function and the MSE for each epoch. The MSE is a common metric to evaluate regession task. The lower the loss, the better the model performance. The purpose of the loss function is to evaluate how well the model is learning the patterns in the data and adjusting its weights and biases to make accurate predictions. So, when you train a model, you try to minimize the loss function by adjusting the model's parameters. In other evaluation metrics, such as accuracy or F1 score, the higher the value, the better the model performance.
Ploting the training history:

In [ ]:
plt.figure(figsize=(16,5))
plt.title('arpeggio')
plt.subplot(1,2,1)

# summarize history for accuracy
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('Model MSE')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')

plt.subplot(1,2,2)
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

## Model Evaluation 
Testing the model whit a testing set computing the loss function for an unseen dataset. 
The testing dataset have to be useen data but whith a trained target set, you can use a dataset belonging to a target that you did not train. In the training the network see the label, in the tasting set it see the label only at the end, but shold be data from a studied label otherwise it has no sense. 
it has to be data belonging to one of the category which you use during the training but unseen during the training stage.
you have to split the training dataset to obtai a testing dataset. 


In [9]:
## Model evaluation 
model.evaluate(X_test,Y_test)

NameError: name 'X_test' is not defined

## Prediction of new data
Now we have to actualy try the regression, we could use the method model.predict

In [ ]:
# Reshape the data to match the input shape of the model
new_data= pd_dataframe['bending'][2]

new_data= np.reshape(new_data,(1,new_data.shape[0], new_data.shape[1]))
print(new_data.shape)
#new_data = new_data.reshape(1, -1)

# Make a prediction using the model
prediction = model.predict(new_data)
print(prediction, prediction.shape)

## Exporting the model 
Now we want to save the model to use it in RawPower

In [ ]:
# Save the model
model.save('sEMG_regression.h5')

Then we could load again the model from everywhere else, be sure to import the same libraries imported at the beginning.

In [5]:
# Load the model
model = load_model('sEMG_regression.h5')

In [8]:
# Reshape the data to match the input shape of the model
new_data= pd_dataframe['bending'][2]

new_data= np.reshape(new_data,(1,new_data.shape[0], new_data.shape[1]))
print(new_data.shape)
#new_data = new_data.reshape(1, -1)

# Make a prediction using the model
prediction = model.predict(new_data)
print(prediction, prediction.shape)

(1, 1, 8)
1/1 [==============================] - 1s 551ms/step
[[0.4121541  0.32937828 0.86067164 0.7390499  0.23566648]] (1, 5)
